In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../")

import pandas as pd
import time
from azureml.core import Workspace
ws = Workspace.from_config()

kv=ws.get_default_keyvault()


In [ ]:
from aml_obs.query import RT_Visualization
streaming_table_name="streaming_test"

rt_viz =RT_Visualization(streaming_table_name,ws)
rt_viz.scatter(max_records=200, ago='12h',groupby='lr', y_metric='metric1',x_metric='timestamp')

### Anomaly Detection

In [ ]:
from aml_obs.query import KustoQuery
query = KustoQuery(streaming_table_name, ws)

query.anomaly_detection(min_t="4/26/2022 10:08:45", max_t="4/23/2022 10:10:00", step="0.5s", metric="metric1", agg="avg", ts_col="timestamp", groupby="lr", sensitivity=0.1, filter="0.001")


### Drift Query

In [7]:
from aml_obs.drift_analysis import Drift_Analysis
drift_analysis =Drift_Analysis(ws)
#test1 1m 1m28 vs 2m26
#test2 10m 1459 vs 10m59


In [8]:
drift_analysis.sample_data_points("StormEvents", col_name="EventId", start_date = "01/01/2007", horizon=30 )

,EventId
0,13208
1,23358
2,23357
3,9494
4,9488
...,...
3261,9963
3262,12466
3263,9969
3264,11485


In [ ]:
df_output = drift_analysis.analyze_drift(limit=10000,base_table_name = 'ISDWeather',target_table_name='ISDWeather', base_dt_from='2013-04-13', base_dt_to='2014-05-13', target_dt_from='2013-04-13', target_dt_to='2014-05-13', bin='30d')


In [ ]:
feature_distribution = drift_analysis.get_features_distributions(target_table_name='ISDWeather',target_dt_from='2013-04-13', target_dt_to='2014-05-13', bin='30d')